In [0]:
import numpy as np
import scipy as sp
from scipy.misc import derivative

In [0]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))
mse = lambda y, preds: ((y - preds)**2)

In [0]:
dersigm = lambda x: ((1 - x) * x)

In [0]:
class Neuron():
    def __init__(self, actfunc=sigmoid, number=0):
        self.LN = actfunc
        self.w = None
        self.inp = None
        self.delta = None
        self.nout = None
        self.nin = []
        self.number = number
    def forward(self, inp):
        self.inp = inp
        if type(self.w) == type(None):
            self.w = np.random.rand(inp.shape[0])
        
        self.out = np.dot(self.w, self.inp)

        return self.LN(self.out)
    def addoutneuron(self, lay):
        self.nout = lay
    def addinneuron(self, neurons):
        self.nin += neurons
    def countdelta(self, answer, nextlay, isoutlay=False):
        
        derivLN = lambda out: (1 - out)*out if self.LN == sigmoid else derivative(self.LN, out)
        
        if isoutlay:
            self.delta = (answer - self.forward(self.backlog())) * derivLN(self.forward(self.backlog())) 
            #Maybe I need to use self.backlog as argument to deriv
        else:
            self.delta = derivLN(self.forward(self.backlog())) * sum([neuron.w[self.number] * neuron.delta for neuron in nextlay.neurons])
        return self.delta
    
    def backlog(self):
        return self.inp

In [0]:
class perceptronlayer():
    def __init__(self, n_counts, actfunc=sigmoid):
        self.neurons = [Neuron(actfunc=actfunc, number=i) for i in range(n_counts)]
        self.size = n_counts
    
    def output(self):
        return np.array([neuron.forward(neuron.inp) for neuron in self.neurons])

    def takeinput(self, inp):
        for neuron in self.neurons:
            neuron.forward(inp)
    def takeweights(self):
        return np.array([neuron.w for neuron in self.neurons])

In [0]:
class Perceptron():
    
    def __init__(self):
        self.lays = []
    
    def addlay(self, n_counts, actfunc=sigmoid):
        self.lays += [perceptronlayer(n_counts, actfunc)]
        if len(self.lays) != 1:
            for neuron in self.lays[-2].neurons:
                neuron.addoutneuron(self.lays[-1])
    
    def predict1(self, X):
        i = 0
        for lay in self.lays:
            if i == 0:
                lay.takeinput(X)
                out = lay.output()
            else:
                lay.takeinput(out)
                out = lay.output()
            i += 1
        return np.exp(out)/sum(np.exp(out))
    
    def fit1(self, X, y, e=0.07):
        self.predict1(X)
        for layer in self.lays[::-1]:
            if layer == self.lays[-1]:
                for neuron in layer.neurons:
                    neuron.countdelta(y, None, True)
            elif layer == self.lays[0]:
                for neuron in range(len(layer.neurons)):
                    layer.neurons[neuron].w = layer.neurons[neuron].w.fill(1)
            else:
                for neuron in layer.neurons:
                    neuron.countdelta(None, neuron.nout, False)
                    for n in neuron.nout.neurons:
                        grad = n.delta * neuron.forward(neuron.backlog())
                        #print(n.w[neuron.number])
                        n.w[neuron.number] = n.w[neuron.number] + e * grad
                        #print(n.w[neuron.number], grad)

    def fit(self, X, y, e=0.07, num_epochs=10):
        for epoch in range(num_epochs):
            w = [neuron.w for neuron in layer.neurons for layer in self.lays]
            #print(w)
            for i in range(len(y)):
                self.fit1(X[i], y[i], e)
    def predict(self, X):
        return np.array([self.predict1(x) for x in X])

In [310]:
perc = Perceptron()
perc.addlay(2)
perc.addlay(3)
perc.addlay(1)
perc.lays

In [311]:
#generate a random data to predict
datatrain = np.random.rand(100, 5)
trueanstrain = np.array([int(round(sum(x))**2 > 4) for x in datatrain])
datatest = np.random.rand(10, 5)
trueanstest = np.array([int(round(sum(x))**2 > 4) for x in datatest])
print(datatrain, trueanstrain)

[[2.22420446e-01 9.35572378e-01 5.94423788e-02 8.43639605e-01
  5.02602308e-01]
 [4.33112128e-01 8.03147561e-01 1.81336535e-01 5.48884603e-03
  2.62382997e-01]
 [3.15715707e-01 3.61254138e-01 6.89305292e-01 3.86164993e-01
  2.28720237e-02]
 [3.81094938e-01 3.96239133e-01 6.06621200e-01 6.31430144e-01
  3.31699692e-01]
 [1.73453519e-01 1.82203314e-01 8.09723655e-01 2.36431374e-01
  6.32431273e-01]
 [8.87990355e-01 6.30595508e-01 8.60197629e-01 6.42436264e-01
  1.41475402e-01]
 [8.22858627e-01 7.89074059e-01 2.76137111e-01 2.35476958e-01
  4.46406513e-01]
 [8.34549987e-01 5.70229638e-01 3.23277181e-01 1.59598716e-01
  4.88921531e-01]
 [1.62283431e-01 6.74721255e-01 2.15313452e-01 6.42093368e-01
  8.51884034e-01]
 [9.29149702e-01 7.17663055e-02 8.16070287e-01 3.16291434e-01
  6.31823199e-01]
 [3.41388351e-01 5.78783892e-01 4.91259261e-01 1.50788485e-01
  5.40570970e-01]
 [3.84569854e-01 3.68893484e-01 6.78902257e-01 5.83657023e-01
  5.00510039e-02]
 [8.02396324e-02 7.54270358e-01 6.227218

In [312]:
sum(perc.predict(datatest).reshape(10) == trueanstest) / 10

0.7

In [0]:
perc.fit(datatrain, trueanstrain, 0.7, num_epochs=10)

In [314]:
sum(perc.predict(datatest).reshape(10) == trueanstest) / 10

0.7